In [10]:
import os
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import torchvision.models as models
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

In [ ]:
# 1. Setup & Configuration
DATA_DIR = "/kaggle/input/wikiart"
selected_classes = ["Impressionism", "Realism", "Expressionism", "Romanticism", "Post_Impressionism"]
NUM_CLASSES = len(selected_classes) # to use in the last layer in the model
BATCH_SIZE = 32
EPOCHS = 13  
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
# 2. Dataset Preparation (FIXED SPLIT)
# Basic transform to read data initially
base_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor() # convert image to tensor => to pytorch understand him
])

In [13]:
# Load full dataset
full_dataset = datasets.ImageFolder(DATA_DIR, transform=base_transform)

# Filter for selected classes
selected_idx = [full_dataset.class_to_idx[c] for c in selected_classes]
filtered_samples = [(path, selected_idx.index(label))
                    for path, label in full_dataset.samples if label in selected_idx]

# Extract labels for stratified split
all_labels = [label for _, label in filtered_samples]

print(f"Total filtered images: {len(filtered_samples)}")


Total filtered images: 43998


In [ ]:
# --- FIX: Proper Train / Val / Test Split ---
train_val_samples, test_samples, train_val_labels, test_labels = train_test_split(
    filtered_samples, all_labels, test_size=0.2, random_state=42, stratify=all_labels
)

train_samples, val_samples = train_test_split(
    train_val_samples, test_size=0.2, random_state=42, stratify=train_val_labels
)

print(f"Train size: {len(train_samples)}")
print(f"Val size:   {len(val_samples)}")
print(f"Test size:  {len(test_samples)}")

# Custom Dataset Class
class FilteredWikiArt(Dataset):
    def __init__(self, samples, transform=None):
        self.samples = samples
        self.transform = transform

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        img = Image.open(path).convert("RGB") # RGB => because efficient pretrained on rgb images not grayscale (3,224,224)
        if self.transform:
            img = self.transform(img)
        return img, label


Train size: 28158
Val size:   7040
Test size:  8800


In [15]:
# 3. Augmentations & Loaders
train_tfms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
])

val_tfms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
])


In [ ]:
# 4. Weighted Sampling for Imbalanced Classes

# Compute class counts
from collections import Counter
train_labels_only = [label for _, label in train_samples]
class_counts = Counter(train_labels_only)
class_weights = [1.0 / class_counts[i] for i in range(NUM_CLASSES)]
sample_weights = [class_weights[label] for label in train_labels_only]

from torch.utils.data import WeightedRandomSampler
sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True) # the images that weigh little => pull them a lot

In [17]:
train_ds = FilteredWikiArt(train_samples, transform=train_tfms)
val_ds = FilteredWikiArt(val_samples, transform=val_tfms)
test_ds = FilteredWikiArt(test_samples, transform=val_tfms)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, sampler=sampler, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)


In [18]:
model = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, NUM_CLASSES)
model = model.to(device)

criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

optimizer = optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-4)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

scaler = torch.cuda.amp.GradScaler(enabled=(device.type == "cuda"))

/tmp/ipykernel_47/1164337018.py:17: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(device.type == "cuda"))


In [ ]:
# 5. Training Loop
best_val_acc = 0.0
patience = 5  # for early stopping 
no_improve = 0
ckpt_path = "efficientnet_b0_best.pth"

print("\nStarting Training...")
for epoch in range(EPOCHS):
    model.train()
    running_loss, running_correct, seen = 0.0, 0, 0
    
    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS} [Train]")
    for imgs, labels in loop:
        imgs, labels = imgs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        # Mixed Precision Training
        with torch.amp.autocast('cuda', enabled=(device.type == "cuda")):
            outputs = model(imgs)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item() * imgs.size(0)
        running_correct += (outputs.argmax(1) == labels).sum().item()
        seen += imgs.size(0)
        
        loop.set_postfix(loss=loss.item())

    # Step the scheduler
    scheduler.step()
    
    train_loss = running_loss / seen
    train_acc = running_correct / seen

    # Validation
    model.eval()
    val_correct, val_seen = 0, 0
    
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            val_correct += (outputs.argmax(1) == labels).sum().item()
            val_seen += labels.size(0)

    val_acc = val_correct / val_seen
    
    print(f"Epoch {epoch+1} Results: Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f} | LR: {optimizer.param_groups[0]['lr']:.6f}")

    # Checkpointing
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), ckpt_path)
        no_improve = 0
        print(">>> New best model saved!")
    else:
        no_improve += 1
        print(f"No improvement for {no_improve} epochs.")
        if no_improve >= patience:
            print("Early stopping triggered.")
            break



Starting Training...


Epoch 1/13 [Train]: 100%|██████████| 880/880 [07:30<00:00,  1.96it/s, loss=1.18] 


Epoch 1 Results: Train Acc: 0.5485 | Val Acc: 0.5879 | LR: 0.000296
>>> New best model saved!


Epoch 2/13 [Train]: 100%|██████████| 880/880 [07:31<00:00,  1.95it/s, loss=1.19] 


Epoch 2 Results: Train Acc: 0.6138 | Val Acc: 0.6412 | LR: 0.000283
>>> New best model saved!


...

Epoch 11 Results: Train Acc: 0.7729 | Val Acc: 0.7151 | LR: 0.000017
>>> New best model saved!


Epoch 12/13 [Train]: 100%|██████████| 880/880 [07:22<00:00,  1.99it/s, loss=0.796]


Epoch 12 Results: Train Acc: 0.7785 | Val Acc: 0.7173 | LR: 0.000004
>>> New best model saved!


Epoch 13/13 [Train]: 100%|██████████| 880/880 [07:28<00:00,  1.96it/s, loss=0.798]


Epoch 13 Results: Train Acc: 0.7821 | Val Acc: 0.7169 | LR: 0.000000
No improvement for 1 epochs.
